## Libraries

In [2]:
#import pandas numpy and scipy libraries
import pandas as pd
import numpy as np
import scipy


In [3]:
# import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Import Data

In [7]:
df_segmentation = pd.read_csv('segmentation data.csv')